# 📊 Tutorial SAEV - Relatórios e Análises Avançadas

Este notebook demonstra como usar as funcionalidades avançadas do Sistema de Análise de Avaliações Educacionais (SAEV).

## 🎯 O que você aprenderá:

1. **Como gerar relatórios automatizados** - Relatórios municipais em Excel
2. **Como fazer análises de clustering** - Agrupamento de escolas por perfil
3. **Como usar análises estatísticas** - Equidade educacional e correlações
4. **Como criar visualizações** - Gráficos interativos dos resultados

## ⚠️ Requisitos:
- Banco de dados SAEV configurado (teste ou produção)
- Dependências instaladas: `pip install -r requirements.txt`

---

## 1️⃣ Importar Bibliotecas e Configurar Conexão

Primeiro, vamos importar todas as bibliotecas necessárias e configurar a conexão com o banco de dados.

In [ ]:
# Importar bibliotecas básicas
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Configurar o caminho para importar módulos do SAEV
sys.path.append(str(Path.cwd()))

# Importar módulos específicos do SAEV
from src.reports.generator import SAEVReports
from src.analytics.advanced import SAEVAnalytics
from src.config import config

# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")

print("✅ Bibliotecas importadas com sucesso!")
print(f"📁 Diretório atual: {Path.cwd()}")

In [ ]:
# Verificar ambiente e configurar banco de dados
print("🔍 Verificando ambientes disponíveis...")

# Listar ambientes disponíveis
environments = config.list_available_environments()

for env_name, info in environments.items():
    status = "✅ Disponível" if info.get('database_exists', False) else "❌ Indisponível"
    print(f"🗃️  {env_name.upper()}: {status}")
    if info.get('database_exists', False) and 'file_size_mb' in info:
        print(f"   📦 Tamanho: {info['file_size_mb']} MB")

# Detectar ambiente atual
current_env = config.detect_environment()
db_path = config.get_database_path(current_env)

print(f"\n🎯 Ambiente detectado: {current_env.upper()}")
print(f"📁 Banco de dados: {db_path}")

# Verificar se há dados
import sqlite3
conn = sqlite3.connect(db_path)
cursor = conn.execute("SELECT COUNT(*) FROM avaliacao")
total_records = cursor.fetchone()[0]
conn.close()

print(f"📊 Total de registros: {total_records:,}")

if total_records == 0:
    print("⚠️ Nenhum dado encontrado! Execute um script de carga primeiro.")
else:
    print("✅ Dados encontrados! Pronto para análises.")

## 2️⃣ Gerar Relatório Municipal

Agora vamos aprender como gerar relatórios municipais automatizados. Estes relatórios incluem:
- Taxa de acerto por município
- Número de alunos e escolas
- Classificação de performance
- Estatísticas resumidas

In [ ]:
# EXEMPLO 1: Método mais simples (detecção automática do banco)
print("📋 Gerando Relatório Municipal - Método Simples")
print("="*50)

# Criar instância do gerador de relatórios (sem especificar banco)
reports = SAEVReports()  # ← Detecção automática!

try:
    # Gerar relatório municipal para Matemática em 2023
    arquivo_relatorio = reports.generate_municipal_report(2023, "Matemática")
    print(f"✅ Relatório gerado com sucesso!")
    print(f"📁 Arquivo salvo em: {arquivo_relatorio}")
    
    # Mostrar informações do arquivo
    arquivo_path = Path(arquivo_relatorio)
    if arquivo_path.exists():
        tamanho_kb = arquivo_path.stat().st_size / 1024
        print(f"📦 Tamanho do arquivo: {tamanho_kb:.1f} KB")
    
except Exception as e:
    print(f"❌ Erro ao gerar relatório: {e}")
    print("💡 Verifique se há dados para o ano e disciplina especificados")

In [ ]:
# EXEMPLO 2: Método avançado (especificando ambiente)
print("\n📋 Gerando Relatório Municipal - Método Avançado")
print("="*50)

try:
    # Especificar banco de teste explicitamente
    db_teste = config.get_database_path('teste')
    reports_teste = SAEVReports(db_teste)
    
    print(f"🗃️  Usando banco: {db_teste}")
    
    # Gerar múltiplos relatórios
    disciplinas = ["Matemática", "Português"]
    arquivos_gerados = []
    
    for disciplina in disciplinas:
        try:
            arquivo = reports_teste.generate_municipal_report(2023, disciplina)
            arquivos_gerados.append(arquivo)
            print(f"✅ {disciplina}: {Path(arquivo).name}")
        except Exception as e:
            print(f"⚠️ {disciplina}: Erro - {e}")
    
    print(f"\n📂 Total de relatórios gerados: {len(arquivos_gerados)}")
    
except Exception as e:
    print(f"❌ Erro geral: {e}")

## 3️⃣ Análise de Clustering de Escolas

O clustering agrupa escolas com características similares de desempenho. Isso nos ajuda a:
- Identificar padrões de performance
- Agrupar escolas por perfil (alto/médio/baixo desempenho)
- Descobrir escolas que precisam de atenção especial

In [ ]:
# EXEMPLO: Clustering de Escolas
print("🔬 Realizando Clustering de Escolas")
print("="*40)

# Criar instância do analisador (detecção automática)
analytics = SAEVAnalytics()  # ← Detecção automática!

try:
    # Fazer clustering para Matemática em 2023
    resultado_clustering = analytics.school_clustering(2023, "Matemática")
    
    if 'error' not in resultado_clustering:
        print(f"✅ Clustering concluído com sucesso!")
        print(f"📊 Número de grupos identificados: {resultado_clustering['n_clusters']}")
        print(f"🏫 Total de escolas analisadas: {len(resultado_clustering['data'])}")
        
        # Mostrar informações de cada cluster
        print("\n📋 Grupos identificados:")
        for cluster_id, nome_cluster in resultado_clustering['cluster_names'].items():
            escolas_no_cluster = len(resultado_clustering['data'][
                resultado_clustering['data']['cluster'] == cluster_id
            ])
            print(f"   🏷️  Grupo {cluster_id + 1}: {nome_cluster}")
            print(f"      📊 Número de escolas: {escolas_no_cluster}")
        
        # Salvar dados do clustering em uma variável para uso posterior
        dados_clustering = resultado_clustering['data']
        print(f"\n💾 Dados salvos na variável 'dados_clustering'")
        
    else:
        print(f"⚠️ {resultado_clustering['error']}")
        print("💡 Isso pode acontecer se não houver dados suficientes para clustering")
        
        # Vamos tentar algo mais simples
        print("\n🔄 Tentando análise de equidade como alternativa...")
        
except Exception as e:
    print(f"❌ Erro no clustering: {e}")
    print("💡 Verifique se há dados suficientes no banco")

In [ ]:
# ANÁLISE DE EQUIDADE EDUCACIONAL
print("⚖️ Analisando Equidade Educacional")
print("="*40)

try:
    # Análise de equidade para Português em 2023
    resultado_equidade = analytics.equity_analysis(2023, "Português")
    
    print(f"✅ Análise de equidade concluída!")
    print(f"🏘️ Municípios analisados: {len(resultado_equidade)}")
    
    # Mostrar os 3 municípios com melhor equidade (menor variação)
    melhor_equidade = resultado_equidade.nsmallest(3, 'coef_variacao')
    print(f"\n🏆 Top 3 - Municípios com MELHOR equidade:")
    for i, (_, row) in enumerate(melhor_equidade.iterrows(), 1):
        print(f"   {i}. {row['MUN_NOME']}")
        print(f"      📊 Nível: {row['nivel_equidade']}")
        print(f"      📈 Coef. Variação: {row['coef_variacao']:.1f}%")
        print(f"      🏫 Escolas: {row['num_escolas']}")
    
    # Mostrar os 3 que mais precisam de atenção
    pior_equidade = resultado_equidade.nlargest(3, 'coef_variacao')
    print(f"\n⚠️ Municípios que precisam de ATENÇÃO:")
    for i, (_, row) in enumerate(pior_equidade.iterrows(), 1):
        print(f"   {i}. {row['MUN_NOME']}")
        print(f"      📊 Nível: {row['nivel_equidade']}")
        print(f"      📈 Coef. Variação: {row['coef_variacao']:.1f}%")
    
    # Salvar dados para visualização
    dados_equidade = resultado_equidade
    print(f"\n💾 Dados salvos na variável 'dados_equidade'")
    
except Exception as e:
    print(f"❌ Erro na análise de equidade: {e}")

## 4️⃣ Visualizar Resultados

Agora vamos criar gráficos interativos para visualizar os resultados das nossas análises.

In [ ]:
# VISUALIZAÇÕES AVANÇADAS
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Instanciar o sistema de análise
analytics = SAEVAnalytics(config_obj=config)

# 1. GRÁFICO DE DISPERSÃO - Português vs Matemática por Grupo
df_groups = analytics.clustering_analysis(n_clusters=4)
fig_scatter = px.scatter(
    df_groups,
    x='MEDIA_PORTUGUES', 
    y='MEDIA_MATEMATICA',
    color='cluster',
    size='TOTAL_ALUNOS',
    hover_data=['MUNICIPIO', 'QTD_ESCOLAS'],
    title="📊 Análise de Agrupamento: Português vs Matemática",
    labels={
        'MEDIA_PORTUGUES': 'Média de Português',
        'MEDIA_MATEMATICA': 'Média de Matemática',
        'cluster': 'Grupo'
    }
)
fig_scatter.show()

# 2. HEATMAP DE CORRELAÇÃO
correlation_data = analytics.correlation_analysis()
fig_heatmap = px.imshow(
    correlation_data['correlation_matrix'],
    title="🔥 Mapa de Calor - Correlações entre Indicadores",
    color_continuous_scale='RdBu',
    aspect='auto'
)
fig_heatmap.show()

# 3. RANKING DOS MUNICÍPIOS
equity_analysis = analytics.equity_analysis()
top_10 = equity_analysis['top_municipalities'][:10]
fig_ranking = px.bar(
    x=top_10['MUNICIPIO'],
    y=top_10['MEDIA_GERAL'],
    title="🏆 Top 10 Municípios - Melhor Desempenho",
    labels={'x': 'Município', 'y': 'Média Geral'}
)
fig_ranking.update_xaxes(tickangle=45)
fig_ranking.show()

print("✅ Visualizações geradas com sucesso!")

## 5️⃣ Comparação de Métricas

Vamos comparar diferentes indicadores educacionais entre municípios.

In [ ]:
# ANÁLISE COMPARATIVA DETALHADA

# 1. DASHBOARD MUNICIPAL INTERATIVO
print("🚀 Iniciando o dashboard interativo...")
print("Aguarde alguns segundos para o dashboard abrir no seu navegador.\n")

# Para executar o dashboard, use o script iniciar.sh ou:
# streamlit run src/dashboard/main.py

# 2. ANÁLISE DE TENDÊNCIAS (se você tiver dados históricos)
try:
    trend_data = analytics.trend_analysis(years=[2023])
    print("📈 Análise de tendências disponível:")
    for metric, value in trend_data['summary_stats'].items():
        print(f"   • {metric}: {value:.2f}")
except Exception as e:
    print(f"⚠️  Análise de tendências requer dados históricos: {e}")

# 3. RELATÓRIO ESTATÍSTICO COMPLETO
print("\n📊 ESTATÍSTICAS GERAIS DO SISTEMA:")
print("="*50)

# Carregar dados básicos
processor = SAEVDataProcessor(config_obj=config)
dados = processor.load_evaluation_data()

if not dados.empty:
    print(f"📍 Total de municípios: {dados['MUNICIPIO'].nunique()}")
    print(f"🏫 Total de escolas: {dados['QTD_ESCOLAS'].sum()}")
    print(f"👥 Total de alunos: {dados['TOTAL_ALUNOS'].sum():,}")
    print(f"📚 Média geral Português: {dados['MEDIA_PORTUGUES'].mean():.2f}")
    print(f"🔢 Média geral Matemática: {dados['MEDIA_MATEMATICA'].mean():.2f}")
    
    # Estatísticas de distribuição
    print(f"\n📊 DISTRIBUIÇÃO DE DESEMPENHO:")
    print(f"   • Melhor município (Português): {dados.loc[dados['MEDIA_PORTUGUES'].idxmax(), 'MUNICIPIO']} ({dados['MEDIA_PORTUGUES'].max():.2f})")
    print(f"   • Melhor município (Matemática): {dados.loc[dados['MEDIA_MATEMATICA'].idxmax(), 'MUNICIPIO']} ({dados['MEDIA_MATEMATICA'].max():.2f})")
    print(f"   • Desvio padrão Português: {dados['MEDIA_PORTUGUES'].std():.2f}")
    print(f"   • Desvio padrão Matemática: {dados['MEDIA_MATEMATICA'].std():.2f}")
    
else:
    print("❌ Não foi possível carregar os dados para análise.")

print("\n" + "="*50)
print("✅ Tutorial concluído com sucesso!")

## 🎯 Próximos Passos

### Como usar o sistema completo:

1. **Dashboard Interativo**: Execute `./iniciar.sh` e escolha a opção dashboard
2. **Relatórios Automáticos**: Use os scripts `demo_funcionalidades.py` ou `exemplo_simples.py`
3. **Análises Personalizadas**: Modifique este notebook para suas necessidades específicas

### Arquivos importantes:
- `src/config.py` - Configuração do ambiente
- `src/reports/generator.py` - Geração de relatórios Excel
- `src/analytics/advanced.py` - Análises estatísticas avançadas
- `src/dashboard/main.py` - Interface web Streamlit

### Dicas:
- Use o ambiente de teste para experimentar (`iniciar_teste.sh`)
- Todos os relatórios são salvos na pasta `reports/`
- O sistema detecta automaticamente qual banco de dados usar

---
**🚀 Sistema SAEV pronto para uso!**